In [95]:
#Imports
from flask import Flask, request
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os
from PIL import Image
import base64

<h3>Loading the model :<h3/>

In [97]:
def load_model(nb_features,loss, optimizer='adam', metrics=['accuracy']):
    resnet_features = ResNet50(include_top=False, input_shape=(112, 112, 3), pooling='avg')
    for layer in resnet_features.layers:
        layer.trainable = False
    model = Sequential([resnet_features,Dense(nb_features),Activation('sigmoid') ])
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)  
    model = keras.models.load_model(glob.glob('./model_20_epochs.h5')[0])
    return model

model = load_model(40,'binary_crossentropy')

<h3>Preparing the testing dataset :<h3/>

In [98]:
DATASET_PATH = './input/list_attr_celeba.csv'
IMGS_PATH = './input/img_align_celeba/img_align_celeba'

attr_df = pd.read_csv(DATASET_PATH)

attr_np = np.array(attr_df) #shape (202599,41)

# Making the dataset onehot
result = np.copy(attr_np)
result[result == -1] = 0
celeb_dataset = result

#splitting dataset
cut_size = int(0.2 * len(celeb_dataset))
cut_indices = np.random.randint(0, len(celeb_dataset), size=(cut_size, ))
    
other = celeb_dataset[cut_indices]
main = np.delete(celeb_dataset, cut_indices, axis=0)

train_np, test_np = split_dataset(celeb_dataset, 0.1)

def load_and_process_image(name, attrs):
    full_path = IMGS_PATH + os.sep + name
    image = tf.io.read_file(full_path)
    image = tf.io.decode_jpeg(contents=image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.divide(image, 255.)
    image = tf.image.resize(image, size=(112, 112))

    return image, attrs

img_path = test_np[:, 0]
img_attrs = test_np[:, 1:]
    
path_ds = tf.data.Dataset.from_tensor_slices(img_path)
attrs_ds = tf.data.Dataset.from_tensor_slices(img_attrs.astype(np.int16))
    
test_ds = tf.data.Dataset.zip((path_ds, attrs_ds))

test_ds = test_ds.map(load_and_process_image)

test_ds = test_ds.shuffle(256)
test_ds = test_ds.batch(64)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)



--------------------------------------
TOTAL:	202599 
FIRST:	183324 
SECOND:	20259
--------------------------------------


<h3>Testing the model :<h3/>

In [77]:
thershold = 0.4
#--------
# A list to stock the accuracy for each label

Acc = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for test_images, test_labels in test_ds : 

    # Predicting Labels
    pred_labels = model.predict(test_images)
    
    # Changing Predicted labels to One-Hot
    pred_labels[pred_labels >= thershold] = 1
    pred_labels[pred_labels < thershold] = 0
    
    for i in range(len(test_labels)) :
        for j in range (40):
            if (pred_labels[i][j] == test_labels[i][j]):
                Acc[j] += 1

2/2 [==============================] - 1s 707ms/step


2/2 [==============================] - 1s 721ms/step


2/2 [==============================] - 1s 123ms/step


In [87]:
Acc1 = [x / 20259 for x in Acc]

In [89]:
glob_acc = 0
for i in Acc1 :
    glob_acc += i
    
glob_acc /= 40

In [90]:
glob_acc

0.8396984056468728

In [91]:
features = ['5_o_Clock_Shadow','Arched_Eyebrows','Attractive','Bags_Under_Eyes','Bald','Bangs',
                'Big_Lips','Big_Nose','Black_Hair','Blond_Hair','Blurry','Brown_Hair','Bushy_Eyebrows',
                'Chubby','Double_Chin','Eyeglasses','Goatee','Gray_Hair','Heavy_Makeup','High_Cheekbones',
                'Male','Mouth_Slightly_Open','Mustache','Narrow_Eyes','No_Beard','Oval_Face','Pale_Skin',
                'Pointy_Nose','Receding_Hairline','Rosy_Cheeks','Sideburns','Smiling','Straight_Hair',
                'Wavy_Hair','Wearing_Earrings','Wearing_Hat','Wearing_Lipstick','Wearing_Necklace',
                'Wearing_Necktie','Young']

In [103]:
acc_dic = {i:0 for i in features}
index = 0
for i in acc_dic :
    acc_dic[i] = Acc1[index]
    index += 1

In [104]:
acc_dic

{'5_o_Clock_Shadow': 0.8835085640949701,
 'Arched_Eyebrows': 0.6879905227306382,
 'Attractive': 0.713806209585863,
 'Bags_Under_Eyes': 0.7911545485956859,
 'Bald': 0.9762081050397354,
 'Bangs': 0.8622340688089244,
 'Big_Lips': 0.5985981539069056,
 'Big_Nose': 0.7740263586554124,
 'Black_Hair': 0.7508267930302581,
 'Blond_Hair': 0.8967866133570265,
 'Blurry': 0.9498000888494003,
 'Brown_Hair': 0.7963374302779012,
 'Bushy_Eyebrows': 0.8590256182437436,
 'Chubby': 0.9421491682708919,
 'Double_Chin': 0.952021323856064,
 'Eyeglasses': 0.9492077595142899,
 'Goatee': 0.9368675650328249,
 'Gray_Hair': 0.9625351695542722,
 'Heavy_Makeup': 0.7902166938150945,
 'High_Cheekbones': 0.7147440643664544,
 'Male': 0.8247198775852708,
 'Mouth_Slightly_Open': 0.6971222666469223,
 'Mustache': 0.9591292758773878,
 'Narrow_Eyes': 0.8776839922997186,
 'No_Beard': 0.839083863961696,
 'Oval_Face': 0.7197788637148922,
 'Pale_Skin': 0.9577965348733896,
 'Pointy_Nose': 0.7206179969396318,
 'Receding_Hairline': 0.